In [ ]:
import tensorflow as tf
import numpy as np
import torch.nn as nn
from tensorflow.keras.datasets import mnist

batch_size = 128 #60000개를 한번에 행렬로 가져올 수 없으니
#128개씩 나눠가져옴

#사용할 중간 레이어들. 256으로 되어있다.
#256으로 되어있을땐 확률이 91%로 나온다.

#만일 히든레이어를 돌리지 않을 경우, 확률은 86%로 떨어지는 것을 알 수 있다.(softmax)
#또한, 히든레이어 노드개수를 1로 설정할 경우엔, 10%대로 대폭 감소한다.
nH1=256
nH2=256
nH3=256

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
#데이터를 [총개수/784, 784]로 변경 
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train/255., x_test/255. #0~1사이로 변경
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

#numpy array나 list를 tensor dataset으로 변환
#이후, 데이터셋을 임의로 배치사이즈만큼 섞음
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size)

#히든레이어 적용
#W는 가중치넣는 배열, b는 bias개수
class ANN(nn.Module) :
  def __init__(self):
    self.W_1 = tf.Variable(tf.random.normal(shape=[784, nH1]))
    self.W_2 = tf.Variable(tf.random.normal(shape=[nH1, nH2]))
    self.W_3 = tf.Variable(tf.random.normal(shape=[nH2, nH3]))
    self.W_Out = tf.Variable(tf.random.normal(shape=[nH3, 10]))
    #self.W_Out = tf.Variable(tf.random.normal(shape=[784, 10]))
    self.b_1 = tf.Variable(tf.random.normal(shape=[nH1]))
    self.b_2 = tf.Variable(tf.random.normal(shape=[nH1]))
    self.b_3 = tf.Variable(tf.random.normal(shape=[nH1]))
    self.b_Out = tf.Variable(tf.random.normal(shape=[10]))
    #self.b_Out = tf.Variable(tf.random.normal(shape=[10]))

  def __call__(self, x) :
    H1_Out = tf.nn.relu(tf.matmul(x, self.W_1) + self.b_1)
    H2_Out = tf.nn.relu(tf.matmul(H1_Out, self.W_2) + self.b_2)
    H3_Out = tf.nn.relu(tf.matmul(H2_Out, self.W_3) + self.b_3)
    Out = tf.matmul(H3_Out, self.W_Out) + self.b_Out
    #Out = tf.matmul(x, self.W_Out) + self.b_Out
    return Out

In [ ]:
ANN_model = ANN()
optimizer = tf.optimizers.Adam(0.01)

#softmax 비용함수(비용 구하기) 상세한건 이전거 참조
def cost_ANN_mnist(x,y) :
  y = tf.cast(y, tf.int64)
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y)
  return tf.reduce_mean(loss)

#tf.argmax 비교 통해, 맞는지 확인
def accuracy(x,y) :
  correct = tf.equal(tf.argmax(x,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
  return accuracy

def train_optimization(x,y) :
  with tf.GradientTape() as g:
    pred = ANN_model(x) #Ann 모델에 x 데이터 삽입
    cost = cost_ANN_mnist(pred, y) #비용 구하기
  gradients = g.gradient(cost, vars(ANN_model).values(), unconnected_gradients=
                         tf.UnconnectedGradients.ZERO)
  #기존 gradients와 다르게 뒤에 unconnected...를 정의함으로써, 연결되지않는 그래디언트로 인한 
  #오류를 제거
  optimizer.apply_gradients(zip(gradients, vars(ANN_model).values()))

#300개씩 가져와 batch_x, batch_y에 넣음
#섞인 train data들을, 300개(+y는 1개씩)가져와 batch_x, y에 넣음
for step, (batch_x, batch_y) in enumerate(train_data.take(300), 1):
  train_optimization(batch_x, batch_y)
  #batch_x 넣고, 러닝 돌린다. 이를 batch_y와 비교해, 손실 구하고
  #정확도 역시 판단.
  if step%10 == 0:
    pred = ANN_model(batch_x)
    loss = cost_ANN_mnist(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print("Step: {}, \t Accuracy: {}, \t Loss:{}".format(step, acc.numpy().flatten(),
                                                         loss.numpy().flatten()))

print('='*30)
print('Model Ttest')
print("Test accuracy: ", accuracy(ANN_model(x_test), y_test).numpy().flatten())
print('=' * 30)

Step: 10, 	 Accuracy: [0.0859375], 	 Loss:[2.561834]
Step: 20, 	 Accuracy: [0.1015625], 	 Loss:[2.3733506]
Step: 30, 	 Accuracy: [0.1171875], 	 Loss:[2.473302]
Step: 40, 	 Accuracy: [0.0859375], 	 Loss:[2.3724947]
Step: 50, 	 Accuracy: [0.0625], 	 Loss:[2.381276]
Step: 60, 	 Accuracy: [0.109375], 	 Loss:[2.350648]
Step: 70, 	 Accuracy: [0.125], 	 Loss:[2.3014336]
Step: 80, 	 Accuracy: [0.078125], 	 Loss:[2.3593125]
Step: 90, 	 Accuracy: [0.0625], 	 Loss:[2.302682]
Step: 100, 	 Accuracy: [0.0859375], 	 Loss:[2.304474]
Step: 110, 	 Accuracy: [0.109375], 	 Loss:[2.324452]
Step: 120, 	 Accuracy: [0.0546875], 	 Loss:[2.3355286]
Step: 130, 	 Accuracy: [0.09375], 	 Loss:[2.3164272]
Step: 140, 	 Accuracy: [0.109375], 	 Loss:[2.307311]
Step: 150, 	 Accuracy: [0.125], 	 Loss:[2.292652]
Step: 160, 	 Accuracy: [0.1328125], 	 Loss:[2.2962117]
Step: 170, 	 Accuracy: [0.0859375], 	 Loss:[2.3022137]
Step: 180, 	 Accuracy: [0.1328125], 	 Loss:[2.2924647]
Step: 190, 	 Accuracy: [0.1328125], 	 Loss:[2.30

In [ ]:
#p.102 Ann PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from torch.autograd import Variable

train_data =datasets.MNIST(root = '../Data', train = True, download = True,
                           transform = transforms.ToTensor())
test_data =datasets.MNIST(root = '../Data', train = True, download = True,
                           transform = transforms.ToTensor())
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(train_data, batch_size=128, shuffle=True)

class ANNModel(nn.Module) :
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(ANNModel, self).__init__()
    self.net = nn.Sequential(nn.Linear(input_dim, hidden_dim), nn.ReLU(),
                             nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
                             nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
                             nn.Linear(hidden_dim, output_dim))
    
    def forward(self, x) :
      return self.net(x)

In [ ]:
model = ANNModel(input_dim = 784, hidden_dim = 256, output_dim = 10)
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
step = 0
loss_list = []
accuracy_list = []

for epoch in range(5):
  for i, (images, labels) in enumerate(train_loader):
    train = Variable(images.view(-1, 784))
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = model(train)
    loss = cost(outputs, labels)
    loss.backward()
    optimizer.step()
    step += 1
    if step % 50 == 0:
      correct = 0
      total = 0
      for images, labels in test_loader:
        test = Vriable(images.view(-1, 784))
        outputs = model(test)
        predicted = torch.max(oututs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
      accuracy = correct.item() / total
      loss_list.append(loss.data.item())
      accuracy_list.append(accuracy)
      if step%100 == 0:
        print("step:{}, \tAccuracy:{}, \t Loss:{}".format(step, accuracy, loss))

NotImplementedError: ignored

In [ ]:
"""
Interpolation: p.170에서, W와 b를 찾는게 목적.(in 딥러닝)
궁극적으로 Y찾는것
이는, 긴 다항식의 답을 연립방정식으로 넣는 것과 유사하다.

LDA(W가 해가 딱 위에 있음. 사람을 구별하기 위해 다르게함)
1이란 군집, 2란 군집 지들끼리 표준편차는 최소화, 단 평균차는 클수록 구별 쉬움
PCA(3차원에서 2차원으로 줄이면, 그림자. 해 위치 W.) 각 벡터를 보며, 가장 큰 벡터 위주

오버피팅 해결법
1) 오토인코딩 : 중요한 것 위주로 사이즈를 줄이고, 디코딩 시엔 다시 복원
2) 드랍아웃: 값 소팅해서, 가장 작은 값들 10%를 0으로 만듬.


"""